# 13. Loop-based backtesting
Advantages:
* The trading logic is straightforward to code.
* You can do way more custom stuff more easily, such as stoplosses and takeprofits. I would have no idea how to implement these in vectorized backtesting.
* It resembles live trading.

Disadvantages:
* Slow
* Possible to introduce look-ahead bias. To avoid this, you should be very careful with which data you access. If a backtest looks very good, cross-check the entry/exits.

I want to make this backtester as simple as possible. However for some things, OOP makes things much easier. We will explain these objects first. However we will use way less OOP than my other github repo QS_Extended. 

All objects and helper functions are in the 'backtester' folder.

In [1]:
from datetime import datetime, date, time
from times import get_market_dates, get_market_calendar, last_trading_date_before
from data import get_data
from backtester.portfolio import Portfolio
import json
import pandas as pd

START_DATE = date(2024, 1, 1)
END_DATE = date(2024, 3, 1)

### The Portfolio object
This simulates a portfolio. When an order gets filled, the values within the portfolio are updated. Also, at the end of the day (or how often we wish), the portfolio logs itself. It also keeps track of all fills.

In [2]:
# INTRADAY BACKTESTER

# Initialize historical scanner, portfolio and calendar
with open("../data/output/dailyscreens/GAP_30PCT.json", "r") as f:
    G30 = json.load(f)

portfolio = Portfolio(START_DATE, initial_capital=10000)
calendar = get_market_calendar("datetime")

# Loop through all days
for day in get_market_dates(START_DATE, END_DATE):
    # Scan for gappers
    gappers_today = G30[day.isoformat()]
    if not gappers_today:
        continue

    # AT MARKET OPEN, calculate gaps and enter positions
    # Get data
    data = {}
    previous_close_dt = calendar.loc[last_trading_date_before(day), "regular_close"]
    market_open_dt = calendar.loc[day, "regular_open"]
    market_close_dt = calendar.loc[day, "regular_close"]
    for gapper in gappers_today:
        data[gapper] = get_data(
            gapper,
            previous_close_dt,
            market_close_dt,
            1,
            columns=["open", "close", "halted"],
        )

    # Calculate gaps
    gap = {}
    for gapper in gappers_today:
        # Get the top 3 largest gappers
        previous_close = data[gapper].loc[previous_close_dt, "close"]
        current_open = data[gapper].loc[market_open_dt, "open"]
        gap[gapper] = current_open / previous_close

    sorted_gap = sorted(gap.items(), key=lambda x: x[1], reverse=True)

    # Market order with 20% of cash
    size = portfolio.current_cash * 0.2
    for gapper in gappers_today:
        current_open = data[gapper].loc[market_open_dt, "open"]
        quantity = int(size / current_open)
        portfolio.update_from_fill(
            dt=market_open_dt,
            side="SELL",
            symbol=gapper,
            fill_quantity=quantity,
            fill_price=current_open,
            fees=5,
        )

    # TODO: Implement stoploss

    # AT MARKET CLOSE: close all positions, update portfolio
    for symbol, quantity in portfolio.current_positions.items():
        if quantity != 0:
            close_price = data[symbol].loc[market_close_dt, "close"]
            action = "SELL" if quantity > 0 else "BUY"
            portfolio.update_from_fill(
                dt=market_close_dt,
                side=action,
                symbol=symbol,
                fill_quantity=abs(quantity),
                fill_price=close_price,
                fees=5,
            )

    portfolio.append_portfolio_log(market_close_dt, data)

In [10]:
portfolio.get_df_from_fills_log()

symbol  side  quantity  fill_price  fees
datetime                                                              
2024-02-01 09:30:00  ELAB-2023-11-21  SELL      1923      1.0400     5
2024-02-01 09:30:00  MINM-2021-07-07  SELL       298      6.7100     5
2024-02-01 15:59:00  ELAB-2023-11-21   BUY      1923      0.8399     5
2024-02-01 15:59:00  MINM-2021-07-07   BUY       298      5.0100     5
2024-02-02 09:30:00  CCTG-2024-01-18  SELL       376      5.7800     5
...                              ...   ...       ...         ...   ...
2024-02-29 15:59:00  COCH-2023-10-02   BUY       298      4.8500     5
2024-02-29 15:59:00  ENVB-2020-12-31   BUY       625      1.4400     5
2024-02-29 15:59:00    JL-2024-01-24   BUY        46      1.9374     5
2024-02-29 15:59:00  SCTL-2022-03-22   BUY      1152      1.0650     5
2024-02-29 15:59:00  VLCN-2021-10-06   BUY       673      1.3000     5

[190 rows x 5 columns]

In [16]:
portfolio_log = portfolio.get_df_from_holdings_log()
portfolio_log.head(10)

equity       cash  positions_value  \
datetime                                                     
2024-02-01           10000.000  10000.000                0   
2024-02-01 15:59:00  10871.392  10871.392                0   
2024-02-02 15:59:00  10369.707  10369.707                0   
2024-02-05 15:59:00  10992.172  10992.172                0   
2024-02-06 15:59:00  11953.092  11953.092                0   
2024-02-07 15:59:00   2371.107   2371.107                0   
2024-02-08 15:59:00   2436.594   2436.594                0   
2024-02-09 15:59:00   2472.824   2472.824                0   
2024-02-12 15:59:00   1914.084   1914.084                0   
2024-02-13 15:59:00   2461.589   2461.589                0   

                                                             positions  \
datetime                                                                 
2024-02-01                                                          {}   
2024-02-01 15:59:00       {'ELAB-2023-11-21': 0, 'MINM-2021-07-07': 0}   
2024-02-02 15:59:00  {'ELAB-2023-11-21': 0, 'MINM-2021-07-07': 0, '...   
2024-02-05 15:59:00  {'ELAB-2023-11-21': 0, 'MINM-2021-07-07': 0, '...   
2024-02-06 15:59:00  {'ELAB-2023-11-21': 0, 'MINM-2021-07-07': 0, '...   
2024-02-07 15:59:00  {'ELAB-2023-11-21': 0, 'MINM-2021-07-07': 0, '...   
2024-02-08 15:59:00  {'ELAB-2023-11-21': 0, 'MINM-2021-07-07': 0, '...   
2024-02-09 15:59:00  {'ELAB-2023-11-21': 0, 'MINM-2021-07-07': 0, '...   
2024-02-12 15:59:00  {'ELAB-2023-11-21': 0, 'MINM-2021-07-07': 0, '...   
2024-02-13 15:59:00  {'ELAB-2023-11-21': 0, 'MINM-2021-07-07': 0, '...   

                     return  return_cum  
datetime                                 
2024-02-01            0.000       0.000  
2024-02-01 15:59:00   8.714       8.714  
2024-02-02 15:59:00  -4.615       3.697  
2024-02-05 15:59:00   6.003       9.922  
2024-02-06 15:59:00   8.742      19.531  
2024-02-07 15:59:00 -80.163     -76.289  
2024-02-08 15:59:00   2.762     -75.634  
2024-02-09 15:59:00   1.487     -75.272  
2024-02-12 15:59:00 -22.595     -80.859  
2024-02-13 15:59:00  28.604     -75.384

In [17]:
from matplotlib import pyplot as plt
plt.plot(portfolio_log.index, portfolio_log.equity)
plt.show()

I am going to show the flexibility and will backtest the following systems:
* Pre-market gap short with 10% stoploss and hold to EOD
* IBS strategy on SPY, with realistic entries (that is, not exactly at market close but 1 minute before)
* Every week, buy the 5 stocks in the T500 that have the largest distance to the moving average